## What is the Zestimate and what is the logerror?

Before diving in to this whole 'Zestimate' thing, it may help to clear up a few ideas, at least as we understand them in pursuing our project's goal.

There are two main values dictating home sale prices:

1.) The Market Value = what the BUYER says the property is worth; and

2.) The Appraised Value = what the BANK says the property is worth.

Banks win 99% of the time because banks have 99% of the money, and that's a good thing - it helps mitigate reality between Buyer and Seller.

I can't demand my house sell for a million dollars when the bank says it's only worth a Happy Meal.  Conversely, the buyer can't buy my home for a Happy Meal when the bank says it's **AT LEAST** worth a Taco Bell Tripleupa Box.  (That took some practice.)

Undertanding this communication gap, Zillow was created in 2006 as a way of providing information to both home-buyers and home-sellers, the end goal being a mutual understanding at the beginning of price negotiations.  One of their flagship offerings is their 'Zestimate,' a constantly-updated and fine-tuned home valution model that is used to predict the market value of a home based on things like '*home facts, location, and market conditions*' (italics are directly from their website, https://www.zillow.com/zestimate/).

While strong and highly durable, the Zestimate is not perfect, even by it's own admission.  From the 'Median Error' section of the Zestimate website: "For most major markets, the Zestimate for on-market homes is within 10% of the final sale price more than 95% of the time."

Plain English: in cities of roughly a million or more people, the difference between Zillow's *predicted* home sale price is 10% different from the home's *actual* sale price.  Not bad, but on a \\$300,000 home, Zestimate can only ballpark a sales price range between \\$270- and \\$330-thousand dollars, a potential dream-crusher for both parties (but don't worry: banks still make out alright).

Because homes are not fiat currencies (they have actual, real value), Zillow can continually improve their model with tangible feedback in hopes of minimizing that error gap.

To see what may be driving this error, we are using what we learned in the Clustering Methodologies section of our Data Science Q-Course.  Instead of the listed 'Mean Error,' we are clustering to determine what is driving the 'logerror' experienced in Zillow's predictive model.  Using logerror (a column from our provided MySQL database) means that we are assuming a distribution underlying Zillow estimates and actual home sale prices. 

### NB:

You may be wondering why we're dealing with California data.  At least we know we were.

Turns out, Texas (and a handful of others) is a non-disclosure state, and the Texas Real Estate Commission - Rulers over all things Texas Real Estate - is under no legal obligation to provide any home sale price information to outside companies like Zillow or RedFin (the Pepsi-cousin to Zillow's Coke). 

Take that for what it's worth, but that leads us to believe the logerror drivers we discover will be unique to the California Zestimate model, and cannot be applied universally without a sacrifice in overall accuracy.



In [1]:
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

np.random.seed(123)

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import numpy as np

import src.acquire
import src.prepare
import src.preprocessing


## Quick Flyover: 

In [2]:
df = src.acquire.get_zillow_data()
df.head()

CSV previously generated at `data/raw/zillow_unprocessed.csv`. Reading in that csv as a DataFrame


,Unnamed: 0,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,4,1970746,10887214,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,6.037124e+13,0.006940,2017-01-01,Central,NaN,NaN,Central,Condominium,NaN,NaN


In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'parcelid', 'airconditioningtypeid',
       'architecturalstyletypeid', 'basementsqft', 'bathroomcnt', 'bedroomcnt',
       'buildingclasstypeid', 'buildingqualitytypeid', 'calculatedbathnbr',
       'decktypeid', 'finishedfloor1squarefeet',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12',
       'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50',
       'finishedsquarefeet6', 'fips', 'fireplacecnt', 'fullbathcnt',
       'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbu

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77575 entries, 0 to 77574
Data columns (total 69 columns):
Unnamed: 0                      77575 non-null int64
id                              77575 non-null int64
parcelid                        77575 non-null int64
airconditioningtypeid           25006 non-null float64
architecturalstyletypeid        206 non-null float64
basementsqft                    50 non-null float64
bathroomcnt                     77575 non-null float64
bedroomcnt                      77575 non-null float64
buildingclasstypeid             15 non-null float64
buildingqualitytypeid           49809 non-null float64
calculatedbathnbr               76960 non-null float64
decktypeid                      614 non-null float64
finishedfloor1squarefeet        6035 non-null float64
calculatedfinishedsquarefeet    77375 non-null float64
finishedsquarefeet12            73920 non-null float64
finishedsquarefeet13            42 non-null float64
finishedsquarefeet15           

In [5]:
pd.set_option('max_rows', None)
df.isnull().sum()

Unnamed: 0                          0
id                                  0
parcelid                            0
airconditioningtypeid           52569
architecturalstyletypeid        77369
basementsqft                    77525
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             77560
buildingqualitytypeid           27766
calculatedbathnbr                 615
decktypeid                      76961
finishedfloor1squarefeet        71540
calculatedfinishedsquarefeet      200
finishedsquarefeet12             3655
finishedsquarefeet13            77533
finishedsquarefeet15            74548
finishedsquarefeet50            71540
finishedsquarefeet6             77189
fips                                0
fireplacecnt                    69288
fullbathcnt                       615
garagecarcnt                    52058
garagetotalsqft                 52058
hashottuborspa                  76036
heatingorsystemtypeid           28005
latitude    

In [6]:
df.shape

(77575, 69)

Sooo... out of 69 columns containing 77,575 rows of data, 20.3% of the columns (14/69) are missing 99.3% (77,000/7,575) of the data.  

According to Zillow, one of the metrics making up Zestimate is customer feedback.  Right off the bat, seems like a good starting point to improving the model's logerror would be to either ask fewer questions, or combine categories, or template the questionnaires they send buyers, sellers, and agents.  .7% of anything is by no means 'reliable' data, nor does it contribute in any way to improving predictie modeling.  

The following function from the prepare.py file cleans up a ton of stuff (and, it should be noted that Shay's function work is **unreal**):

In [7]:
df = src.prepare.wrangle_zillow(df)


    Number of rows dropped:    156
    Number of columns dropped: 34
    


### Though a single line of code, that 'wrangle_zillow' function does the following:

- gets rid of columns taht don't meet the info threshold of 75% across rows and 50% down columns;

- drops the columns like 'buildingqualitytypeid' that we used to join tables from MySQL, as well as repetitive filters ('propertycountylandusecode') and the 'calculatedbathnbr' column that was too similar to 'bathroomcnt';

- takes the null values from several columns and fills them with median values to offset the effect of outliers; and

- changes floats like 'fip' to objects and objects to floats based on how we'll be flexing our math muscles.



In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77419 entries, 0 to 77574
Data columns (total 28 columns):
Unnamed: 0                      77419 non-null int64
id                              77419 non-null int64
parcelid                        77419 non-null int64
bathroomcnt                     77419 non-null float64
bedroomcnt                      77419 non-null float64
calculatedfinishedsquarefeet    77419 non-null object
finishedsquarefeet12            77419 non-null object
fips                            77419 non-null float64
fullbathcnt                     77419 non-null object
latitude                        77419 non-null float64
longitude                       77419 non-null float64
lotsizesquarefeet               77419 non-null object
rawcensustractandblock          77419 non-null float64
regionidcity                    77419 non-null object
regionidcounty                  77419 non-null float64
regionidzip                     77419 non-null object
roomcnt                

In [11]:
df.shape

(77419, 28)

### ^ Mas mejor.###

Now that everything's all cleared up, time to split and scale so that we can send our data to the gym (ie, "train") and check it's weight on the 'scale' to make sure the numbers we change are still within the same scope without having their ranges distorted.

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import cm
from sklearn.model_selection import learning_curve
%matplotlib inline

In [14]:
train, test = train_test_split(df, random_state = 123)

In [ ]:
scaler = MinMaxScaler
num_vars = list(train.select_dtypes("number").columns)

print("The following columns are being scaled: {}")

## Taking a closer look:

In [7]:
# Getting Past the Guardrail:

import warnings
warnings.filterwarnings("ignore")

# Grabbing Gear:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Spelunking

import scipy.stats as stats

# Seeing In the Dark

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve
%matplotlib inline




In [8]:
src.prepare.nulls_by_col(df)

,number_missing_rows,percent_rows_missing
Unnamed: 0,0,0.000000
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52569,0.677654
architecturalstyletypeid,77369,0.997345
basementsqft,77525,0.999355
bathroomcnt,0,0.000000
bedroomcnt,0,0.000000
buildingclasstypeid,77560,0.999807
buildingqualitytypeid,27766,0.357925


In [ ]:
train, test = train_test_split(df, random_state = 123)

#### We know that the values for 'finishedsquarefeet' are different from those in 'fips' or 'yearblt,' so we're scaling using the MinMaxScaler to preserve the shape of any underlying distributions.

In [ ]:
scaler = MinMaxScaler()
num_vars = list(train.select_dtypes("number").columns)
num_vars

In [ ]:
print("We are scaling {}, and {}.".format(",".join(num_vars[:-1]), num_vars[-1]))
train[num_vars] = scaler.fit_transform(train[num_vars])

In [ ]:
src.nulls_by_col(df)

In [ ]:
df2 = df.drop(columns=["taxdelinquencyflag","propertyzoningdesc", "propertycountylandusecode"], axis=1)

In [ ]:
df2.corr(method="pearson")

In [ ]:
corr = df2.corr()

plt.figure(figsize=(20,12))

ax = sns.heatmap(
    corr,
    #annot = True,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

**Takeaway:** lots of columns in there actually show white stripes.  Tells me they don't have a whole lot of information on them and can be dropped in the overall scheme of things.

In [ ]:
df3 = df.drop(columns=["decktypeid", "hashottuborspa", "poolcnt", "pooltypeid2",
                      "pooltypeid10", "pooltypeid7", "storytypeid", "fireplaceflag",
                      "assessmentyear"], axis=1)

In [ ]:
corr = df3.corr()

plt.figure(figsize=(40,20))

ax = sns.heatmap(
    corr,
    #annot = True,
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

Yeah, no.  Waste of time.  Was hoping to see some relationships, but no bueno - ugly.